# CROSS APPLY und OUTER APPLY

## Zeige zu **jedem** Produkt die beiden jüngsten Bestellungen mit BestellungID, Bestelldatum, KundenNr und Firma

Die Lösung mit OUTER APPLY hat die achtfachen Ausführungskosten gegenüber der Lösung mit RANK in der Unterabfrage!

```
Fensterfunktionen sind nur in der SELECT- oder ORDER BY-Klausel zulässig.

```

In [ ]:
-- Lösungsversuch mit CROSS APPLY zeigt nur Produkte mit Bestellungen
SELECT p.ProduktID, p.Produkt, a.BestellungID, a.Bestelldatum, a.KundenNr, a.Firma
FROM Produkt AS p
CROSS APPLY (
   SELECT TOP(2) WITH TIES b.BestellungID, b.Bestelldatum, k.KundenNr, k.Firma
    FROM Bestellung AS b
    JOIN BestellDetail AS d ON b.BestellungID = d.BestellungID
    JOIN Kunde AS k ON b.KundenNr = k.KundenNr
    WHERE d.ProduktID = p.ProduktID
    ORDER BY b.Bestelldatum DESC 
) AS a;

In [ ]:
-- Lösung mit OUTER APPLY
SELECT p.ProduktID, p.Produkt, a.BestellungID, a.Bestelldatum, a.KundenNr, a.Firma
FROM Produkt AS p
OUTER APPLY (
   SELECT TOP(2) WITH TIES b.BestellungID, b.Bestelldatum, k.KundenNr, k.Firma
    FROM Bestellung AS b
    JOIN BestellDetail AS d ON b.BestellungID = d.BestellungID
    JOIN Kunde AS k ON b.KundenNr = k.KundenNr
    WHERE d.ProduktID = p.ProduktID
    ORDER BY b.Bestelldatum DESC 
) AS a;

In [ ]:
-- Nicht zulässig! Fensterfunktionen sind nur in der SELECT- oder ORDER BY-Klausel zulässig.
SELECT p.ProduktID, p.Produkt
    , RANK() OVER(PARTITION BY p.ProduktID ORDER BY b.BestellungID DESC) AS Rang
    , b.BestellungID, b.Bestelldatum, k.KundenNr, k.Firma
FROM Bestellung AS b 
JOIN Kunde AS k ON b.KundenNr = k.KundenNr
JOIN BestellDetail AS d ON b.BestellungID = d.BestellungID
RIGHT JOIN Produkt AS p ON d.ProduktID = p.ProduktID
WHERE RANK() OVER(PARTITION BY p.ProduktID ORDER BY b.BestellungID DESC) <=2

In [ ]:
-- Lösung mit vorangestellter Unterabfrage, auf der gefiltert wird
WITH Sub AS (
    SELECT p.ProduktID, p.Produkt
        , RANK() OVER(PARTITION BY p.ProduktID ORDER BY b.BestellungID DESC) AS Rang
        , b.BestellungID, b.Bestelldatum, k.KundenNr, k.Firma
    FROM Bestellung AS b 
    JOIN Kunde AS k ON b.KundenNr = k.KundenNr
    JOIN BestellDetail AS d ON b.BestellungID = d.BestellungID
    RIGHT JOIN Produkt AS p ON d.ProduktID = p.ProduktID 
)
SELECT ProduktID, Produkt, BestellungID, Bestelldatum, KundenNr, Firma
FROM SUB 
WHERE Rang <= 2;

In [ ]:
-- Unterabfrage in FROM Klausel, statt vorangestellt
SELECT ProduktID, Produkt, BestellungID, Bestelldatum, KundenNr, Firma
FROM (
    SELECT p.ProduktID, p.Produkt
        , RANK() OVER(PARTITION BY p.ProduktID ORDER BY b.BestellungID DESC) AS Rang
        , b.BestellungID, b.Bestelldatum, k.KundenNr, k.Firma
    FROM Bestellung AS b 
    JOIN Kunde AS k ON b.KundenNr = k.KundenNr
    JOIN BestellDetail AS d ON b.BestellungID = d.BestellungID
    RIGHT JOIN Produkt AS p ON d.ProduktID = p.ProduktID 
) AS SUB 
WHERE Rang <= 2;

## OUTER APPLY mit Tabellenwertfunktionen einsetzen

Möchte man für Bequemlichkeit in der Abfrageformulierung Performance-Einbußen in Kauf nehmen?

In [ ]:
-- Erstelle Tabellenwertfunktion für jüngste Bestellungen
CREATE OR ALTER FUNCTION dbo.fn_GetNLetzteProduktBestellungen(
    @ProduktID INT,
    @N INT
) 
RETURNS TABLE
AS
RETURN
   SELECT TOP(@N) WITH TIES b.BestellungID, b.Bestelldatum, k.KundenNr, k.Firma
    FROM Bestellung AS b
    JOIN BestellDetail AS d ON b.BestellungID = d.BestellungID
    JOIN Kunde AS k ON b.KundenNr = k.KundenNr
    WHERE d.ProduktID = @ProduktID
    ORDER BY b.Bestelldatum DESC;

In [ ]:
-- Verwende OUTER APPLY mit einer Tabellenwertfunktion
SELECT p.ProduktID, p.Produkt, a.BestellungID, a.Bestelldatum, a.KundenNr, a.Firma
FROM Produkt AS p
OUTER APPLY dbo.fn_GetNLetzteProduktBestellungen(p.ProduktID, 3) AS a;

## Zeige zu **jedem** Kunden die 3 letzten Produkte, die er bestellt hat

Die Lösung mit OUTER APPLY hat die sechsfachen Ausführungskosten der Lösung mit RANK und Unterabfrage!

Bei mehr Produkten am gleichen Bestelldatum werden auch mehr als 3 Produkte ausgegeben.

In [ ]:
SELECT KundenNr, Firma, sub.ProduktID, sub.Produkt, sub.Bestelldatum, sub.Menge
FROM Kunde AS k
OUTER APPLY (
    SELECT TOP(3) WITH TIES b.Bestelldatum, p.ProduktID, p.Produkt, d.Menge
    FROM Bestellung AS b
    JOIN BestellDetail AS d ON b.BestellungID = d.BestellungID
    JOIN Produkt AS p ON d.ProduktID = p.ProduktID
    WHERE b.KundenNr = k.KundenNr
    ORDER BY b.Bestelldatum DESC
) AS sub;

In [ ]:
WITH Sub AS (
    SELECT k.KundenNr, k.Firma, p.ProduktID, p.Produkt, b.Bestelldatum, d.Menge
        , RANK() OVER(PARTITION BY k.KundenNr ORDER BY b.Bestelldatum DESC) AS Rang
    FROM Kunde AS k
    LEFT JOIN Bestellung AS b ON k.KundenNr = b.KundenNr
    LEFT JOIN BestellDetail AS d ON b.BestellungID = d.BestellungID
    LEFT JOIN Produkt AS p ON d.ProduktID = p.ProduktID
)
SELECT KundenNr, Firma, ProduktID, Produkt, Bestelldatum, Menge
FROM Sub
WHERE Rang <= 3
ORDER BY KundenNr;

## Top 3 Produkte nach Umsatz per Kategorie für ein bestimmtes Jahr

Die Lösung mit OUTER APPLY hat die 5,6-fachen Ausführungskosten gegenüber der Alternativlösung.

In [ ]:
-- TOP 3 Produkte je kategorie nach Umsatz in 2018
SELECT k.Kategorie, Sub.Rang, Sub.ProduktID, Sub.Produkt, Sub.Umsatz
FROM Produktkategorie AS k
OUTER APPLY (
    SELECT TOP(3) WITH TIES RANK() OVER(ORDER BY SUM(d.Menge * d.Verkaufspreis) DESC) AS Rang
        , p.ProduktID, p.Produkt, SUM(d.Menge * d.Verkaufspreis) AS Umsatz 
    FROM Bestellung AS b
    JOIN BestellDetail AS d ON b.BestellungID = d.BestellungID
    JOIN Produkt AS p ON d.ProduktID = p.ProduktID
    WHERE YEAR(b.Bestelldatum) = 2017
        AND p.KategorieID = k.KategorieID -- Korrelation
    GROUP BY p.ProduktID, p.Produkt  
	ORDER BY SUM(d.Menge * d.Verkaufspreis) DESC
) AS Sub
ORDER BY k.Kategorie;

In [ ]:
WITH Sub AS (
    SELECT k.Kategorie
    , RANK() OVER(PARTITION BY k.Kategorie ORDER BY SUM(d.Menge * d.Verkaufspreis) DESC) AS Rang
    , p.ProduktID
    , p.Produkt
    , SUM(d.Menge * d.Verkaufspreis) AS Umsatz
    FROM Bestellung AS b
    JOIN BestellDetail AS d ON b.BestellungID = d.BestellungID
        AND YEAR(b.Bestelldatum) = 2017
    JOIN Produkt AS p ON d.ProduktID = p.ProduktID
    RIGHT JOIN Produktkategorie AS k ON p.KategorieID = k.KategorieID
    GROUP BY k.Kategorie, p.ProduktID, p.Produkt
)
SELECT * 
FROM Sub
WHERE Rang <=3;

## Tabellenwertfunktion mit Parametern zur Vorgabe von Produktkategorie, Jahr und Anzahl von Top Produkten

In [ ]:
DROP FUNCTION IF EXISTS dbo.fn_GetTopNProdukteInKategorieUndJahr;
GO
CREATE FUNCTION 
    dbo.fn_GetTopNProdukteInKategorieUndJahr(
            @Kategorie AS VARCHAR(30), 
            @Jahr AS INT, 
            @NumProd AS INT
) 
RETURNS TABLE
AS
RETURN
WITH Sub AS (
    SELECT k.Kategorie
    , RANK() OVER(PARTITION BY k.Kategorie ORDER BY SUM(d.Menge * d.Verkaufspreis) DESC) AS Rang
    , p.ProduktID
    , p.Produkt
    , SUM(d.Menge * d.Verkaufspreis) AS Umsatz
    FROM Bestellung AS b
    JOIN BestellDetail AS d ON b.BestellungID = d.BestellungID
        AND YEAR(b.Bestelldatum) = @Jahr
    JOIN Produkt AS p ON d.ProduktID = p.ProduktID
    RIGHT JOIN Produktkategorie AS k ON p.KategorieID = k.KategorieID
    WHERE k.Kategorie = @Kategorie
    GROUP BY k.Kategorie, p.ProduktID, p.Produkt
)
SELECT * 
FROM Sub
WHERE Rang <= @NumProd;

In [ ]:
-- Top 5 Produkte für Getränke in 2017
SELECT * FROM dbo.fn_GetTopNProdukteInKategorieUndJahr('Getränke', 2017, 5);

In [ ]:
-- Die Top Süßware in 2016
SELECT * FROM dbo.fn_GetTopNProdukteInKategorieUndJahr('Süßwaren', 2016, 1);

## Zwischenergebnisse durch Gruppierung mit ROLLUP

Zeige Umsätze nach Produktkategorie, Produkt, Jahr und Quartal mit Zwischensummen für Jahre, Produkte und Kategorien.

In [ ]:
SELECT k.Kategorie, p.Produkt, DATEPART(YEAR, b.Bestelldatum) AS Jahr
    , DATEPART(QUARTER, b.Bestelldatum) AS Quartal
    , SUM(d.Menge * d.Verkaufspreis) AS Umsatz
FROM Produktkategorie AS k
LEFT JOIN Produkt AS p ON k.KategorieID = p.KategorieID
LEFT JOIN BestellDetail AS d ON p.ProduktID = d.ProduktID
LEFT JOIN Bestellung AS b ON d.BestellungID = b.BestellungID
GROUP BY ROLLUP(k.Kategorie, p.Produkt, DATEPART(YEAR, b.Bestelldatum)
    , DATEPART(QUARTER, b.Bestelldatum));

In [ ]:
-- Versuch einer lesefreundlicheren Variante, die NULL mit Text ersetzt
WITH Sub AS (
    SELECT k.Kategorie, p.Produkt, DATEPART(YEAR, b.Bestelldatum) AS Jahr
        , DATEPART(QUARTER, b.Bestelldatum) AS Quartal
        , SUM(d.Menge * d.Verkaufspreis) AS Umsatz
    FROM Produktkategorie AS k
    LEFT JOIN Produkt AS p ON k.KategorieID = p.KategorieID
    LEFT JOIN BestellDetail AS d ON p.ProduktID = d.ProduktID
    LEFT JOIN Bestellung AS b ON d.BestellungID = b.BestellungID
    GROUP BY ROLLUP(k.Kategorie, p.Produkt, DATEPART(YEAR, b.Bestelldatum)
        , DATEPART(QUARTER, b.Bestelldatum))
)
SELECT COALESCE(Kategorie, '——>') AS Kategorie 
    , COALESCE(Produkt, '——>') AS Produkt
    , COALESCE(CAST(Jahr AS CHAR(4)), '——>') AS Jahr
    , CASE 
        WHEN Kategorie IS NULL THEN 'GRAND TOTAL'
        WHEN Produkt IS NULL THEN CONCAT('TOTAL Kategorie ', Kategorie)
        WHEN Jahr IS NULL THEN CONCAT('TOTAL Produkt ', Produkt)
        WHEN Quartal IS NULL THEN CONCAT('TOTAL Jahr ', Jahr)
        ELSE CONCAT('Q', Quartal, ' / ', Jahr) 
      END AS Quartal
    , Umsatz
FROM Sub;

## Kumulierte Summen ("Running Totals")

Kumuliere die Quartalsumsätze für Jahre, gruppiert nach Kategorie und Produkt.

### Referenz:

- [Running Total (databasestar.com)](https:\www.databasestar.com\sql-running-total\) 
- [Running Total (learnsql.com)](https:\learnsql.com\blog\what-is-a-running-total-and-how-to-compute-it-in-sql\)

In [ ]:
WITH Sub AS (
    SELECT k.Kategorie, p.Produkt, DATEPART(YEAR, b.Bestelldatum) AS Jahr
        , DATEPART(QUARTER, b.Bestelldatum) AS Quartal
        , SUM(d.Menge * d.Verkaufspreis) AS Umsatz
    FROM Produktkategorie AS k
    LEFT JOIN Produkt AS p ON k.KategorieID = p.KategorieID
    LEFT JOIN BestellDetail AS d ON p.ProduktID = d.ProduktID
    LEFT JOIN Bestellung AS b ON d.BestellungID = b.BestellungID
    GROUP BY k.Kategorie, p.Produkt, DATEPART(YEAR, b.Bestelldatum), DATEPART(QUARTER, b.Bestelldatum)
)
SELECT Kategorie, Produkt, Jahr, Quartal, Umsatz
    , SUM(Umsatz) OVER (PARTITION BY Kategorie, Produkt, Jahr ORDER BY Jahr, Quartal) AS [Kumulierter Jahresumsatz]
FROM Sub
WHERE Jahr = 2017;